In [ ]:
#hide
!nvidia-smi

Thu Jan 28 01:21:25 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 450.36.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro P5000        On   | 00000000:00:05.0 Off |                  Off |
| 26%   27C    P8     6W / 180W |      1MiB / 16278MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#hide
import sys
if 'google.colab' in sys.modules:
    !pip install -Uqq fastai einops datasets axial_positional_embedding wandb
    !pip install -qq git+git://github.com/arampacha/reformer_fastai.git

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#all_slow

# wikitext-2: n_layers experiment 

In [ ]:
from fastai.text.all import *
from reformer_fastai.all import *

In [ ]:
#Run setup
n_epochs = 10
bs = 1
sl = 2**15
n_layers = 3
seed = 21

## Experiment Tracking

Make sure you have wandb and are logged in:

In [ ]:
# hide
# !wandb login

Load Experiment Tracking with Weights & Biases:

In [ ]:
import wandb

WANDB_NAME = f'n_layers-{n_layers}_wikitext2_sl-{sl}_bs-{bs}_n_eps-{n_epochs}_seed-{seed}'
GROUP = 'n_layers2'
NOTES = 'ReformerLM on wikitext2 sl=16k'
CONFIG = {}
TAGS = ['lm','lsh','wikitext2', 'nlayers']

## Download and Unpack enwik8 Data

Download and unzip wikitext2 data

In [ ]:
from datasets import load_dataset
train_ds = load_dataset('wikitext', name='wikitext-2-raw-v1', split='train')

Reusing dataset wikitext (/root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/47c57a6745aa5ce8e16a5355aaa4039e3aa90d1adad87cef1ad4e0f29e74ac91)


In [ ]:
def splitlines(sample):
    return {'line':[*sample['text'][0].split('\n')]}

In [ ]:
train_ds = train_ds.filter(lambda x: x['text'] != '')
df = train_ds.data.to_pandas()

Loading cached processed dataset at /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/47c57a6745aa5ce8e16a5355aaa4039e3aa90d1adad87cef1ad4e0f29e74ac91/cache-b6939be3e780c155.arrow


## Prepare Data

In [ ]:
btt = ByteTextTokenizer(is_lm=True, add_bos=False, add_eos=False)

In [ ]:
%%time
df['toks'] = df['text'].apply(btt)
df['lens'] = df['toks'].apply(len)
df['lens_cum_sum'] = df.lens.cumsum()

CPU times: user 5.05 s, sys: 121 ms, total: 5.17 s
Wall time: 5.1 s


In [ ]:
train_cutoff = 10_000_000
train_idxs = df.loc[df['lens_cum_sum'] < train_cutoff].index.values
train_idxs = list(range(0, max(train_idxs)))

remaining_idxs = len(df) - max(train_idxs)
validation_idxs = list(range(max(train_idxs), max(train_idxs) + int(remaining_idxs)))

splits = [train_idxs, validation_idxs]

In [ ]:
tfms = [attrgetter("text"), btt]
dsets = Datasets(df, [tfms], splits=splits, dl_type=LMDataLoader)

In [ ]:
%%time
dl_kwargs = [{'lens':df['lens'].values[train_idxs]},
             {'val_lens':df['lens'].values[validation_idxs]}]
dls = dsets.dataloaders(bs=bs, seq_len=sl, dl_kwargs=dl_kwargs, shuffle_train=True, n_workers=2)

CPU times: user 3.16 s, sys: 909 ms, total: 4.07 s
Wall time: 4.07 s


In [ ]:
#collapse_output
dls.show_batch()

,text,text_
0,"In the 1930s , legal restrictions on condoms began to be relaxed . But during this period Fascist Italy and Nazi Germany increased restrictions on condoms ( limited sales as disease preventatives were still allowed ) . During the Depression , condom lines by Schmid gained in popularity . Schmid still used the cement @-@ dipping method of manufacture which had two advantages over the latex variety . Firstly , cement @-@ dipped condoms could be safely used with oil @-@ based lubricants . Secondly , while less comfortable , these older @-@ style rubber condoms could be reused and so were more economical , a valued feature in hard times . More attention was brought to quality issues in the 1930s , and the U.S. Food and Drug Administration began to regulate the quality of condoms sold in the United States . \n = = Credits and personnel","In the 1930s , legal restrictions on condoms began to be relaxed . But during this period Fascist Italy and Nazi Germany increased restrictions on condoms ( limited sales as disease preventatives were still allowed ) . During the Depression , condom lines by Schmid gained in popularity . Schmid still used the cement @-@ dipping method of manufacture which had two advantages over the latex variety . Firstly , cement @-@ dipped condoms could be safely used with oil @-@ based lubricants . Secondly , while less comfortable , these older @-@ style rubber condoms could be reused and so were more economical , a valued feature in hard times . More attention was brought to quality issues in the 1930s , and the U.S. Food and Drug Administration began to regulate the quality of condoms sold in the United States . \n = = Credits and personnel ="


In [ ]:
vocab_sz = btt.vocab_size

In [ ]:
# xb, yb = dls.one_batch()
# xb.shape, yb.shape

In [ ]:
#hide
# del xb, yb
# torch.cuda.empty_cache()

## Training

In [ ]:
#hide_output
wandb.init(reinit=True, project="reformer-fastai", entity="fastai_community", 
           name=WANDB_NAME, group=GROUP, notes=NOTES, tags=TAGS, config=CONFIG)

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: fastai_community (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.15 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [ ]:
pad_id = btt.pad_token_id
config = NLayersConfig(d_model=512, d_ff=2048, ff_chunks=8, n_layers=n_layers, max_seq_len=sl, pad_idx=pad_id, seed=seed)
config

ReformerLM config 
--------------------
vocab_sz        256
d_model         512
n_layers        3
n_heads         8
d_ff            2048
ff_chunks       8
attn_dropout    0.1
ff_dropout      0.1
emb_dropout     0.1
tie_weights     True
causal          True
pos_enc         axial
max_seq_len     32768
axial_shape     None
axial_emb_dims  None
pad_idx         0
prenorm         True
attn_bias       False
bucket_size     64
use_lsh         True
n_hashes        8
rev_thres       0
seed            21

In [ ]:
learn = Learner(dls, ReformerLM.from_config(config),
                loss_func=CrossEntropyLossFlat(ignore_index=pad_id), opt_func=adafactor,
                cbs = [GradientAccumulation(n_acc=8), GradientClip(1.0),
                       PadBatchCallback(bucket_size=config.bucket_size,
                                        val=pad_id, y_val=pad_id),
                       SaveModelCallback(with_opt=True)],
                metrics=[accuracy, perplexity, bpc])#.to_fp16()

In [ ]:
#hide
# learn.lr_find()

In [ ]:
learn.fit(n_epochs, cbs=WandbCallback(log_model=True, log_preds=False))

epoch,train_loss,valid_loss,accuracy,perplexity,bpc,time


In [ ]:
learn.recorder.plot_loss()